# forms 组件

能够完成的事情：
    
   - 渲染html也米娜
   - 校验数据
   - 展示提示信息

## 基本使用

写在views.py文件中
```
from django import forms

class MyForms(forms.Form):
    # username字符串类型最小3位最大8位
    username = forms.CharField(min_length=3, max_length=8, lable="用户名")
    # email字段必须符合邮箱格式 xxx@xx.com
    email = forms.EmailField()
```

## 校验数据

```
from app01 import views
# 1 将待校验的数据组织成字典的形式传入，也可以直接传request.POST
form_obj = views.MyForm({'username':'lll', 'email':'123'})
# 2判断数据是否合法，该方法只有在所有的数据都合法的情况下才会返回True
form_obj.is_valid()
# 3 查看所有校验通过的数据
form_obj.cleaned_data
# 4 查看所有不符合校验规则以及不符合的原因
form_obj.errors
```

校验数据只校验类中出现的字段，多传不影响，多传字段直接忽略

校验数据，默认情况下，类中所有字段都必须传值

form表单取消浏览器自动校验功能：
    
    `<form novalidate></form>`

## 渲染标签

forms组件只会自动帮你渲染获取用户输入的标签（input select radio checkbox）

views.py中

```
def index(request):
    # 1.先产生一个空对象
    form_obj = MyForm()
    # 2. 直接将该空对象传递给html页面
    return render(request, 'index.html', locals())
```

index.html中

```
<form action="" method="post">
    <p>第一种渲染方式:代码书写极少，封装程度高，不便于后续的扩展，一般情况下只在本地测试使用</p>
    {{ form_obj.as_p }}
    
    <p>第二种渲染方式:可扩展性很强，但是需要书写的代码太多</p>
    <p>{{ form_obj.username.lable }}:{{ form_obj.username}}</p>
    
    <p>第三种渲染方式:代码书写简单，并且可扩展性也高</p>
    {% for form in form_obj %}
        <p>{{ form.lable }}:{{ form }}</p>
    {% endfor %}
</form>
```

label属性默认展示是类中定义字段名首字母大写的形式

## 展示提示信息

浏览器会自动帮你校验数据，但是前端的校验弱不经风，需要让浏览器不做校验：
`<form action="" method="post" novalidate>`

html文件

```
<form action="" method="post">
    {% for form in form_obj %}
        <p>{{ form.lable }}:{{ form }}
           <span style="color: red">{{ form.errors.0 }}</span>
        </p>
    {% endfor %}
</form>
```

view.py

```
def index(request):
    # 1 先产生一个空对象
    form_obj = MyForm()
    if request.method == "POST":
        # 校验数据
        form_obj = MyForm(request.POST)  # 此处精髓，与前面form_obj变量同名
        # 判断是否合法
        if form_obj.is_valid():
            return HttpResponse('ok')
        # 不合法
        
    return render(request, 'index.html', locals())
```

总结：
- 必备的条件，get请求和post传给html页面对象的变量名必须一样
- forms组件当你数据不合法的情况下，会保存上次数据，基于之前的数据进行修改
- 提示信息英文修改

```
username = forms.CharField(min_length=3, max_length=8, label='用户名',
    error_messages = {
        'min_length':'用户名最少三位',
        'max_length':'用户名最大八位',
        'required':'用户名不能为空'
    }
)

email = forms.EmailField(label="邮箱",
    error_messages={
        'invalid':"邮箱格式不正确",
        'required':"邮箱不能为空"
    }
)
```

## 钩子函数

在特定节点自动触发完成响应操作

钩子函数在forms组件中就类似于第二道关卡，能够让我们自定义校验规则

forms组件有两类钩子：
    
    - 局部钩子
        
        当你需要给单个字段增加校验规则的时候可以使用
    - 全局钩子
    
        当你需要给多个字段添加校验规则的时候可以使用

### 例子

1. 校验用户名中不能含666， 只是校验username字段  局部钩子

```
class MyForms(forms.Form):
    # 就是这个类中的一个方法
    def clean_username(self):
        username = self.cleaned_data.get('username')  # 走过第一道关卡后才会进入第二道关卡，第一关校验合格的数据都放在cleaned_data中
        if '666' in username:
            # 提示前端展示错误信息
            self.add_error('username', '不能含666')  # 给username字段添加报错信息
            
        # 将钩子函数勾出来的数据再放回去
        return username
```

2. 校验密码和确认密码是否一致，password和confirm两个字段 全局钩子

```
class MyForms(forms.Form):
    def clean(self):
        password = self.cleaned_data.get('password')
        confirm_password = self.cleaned_data.get('confirm_password')
        
        if not confirm_password == password:
            self.add_error('confirm_password', '两次密码不一致')
            
        # 将钩子函数勾出来的数据全部放回去
        return self.cleaned_data
```

## forms组件其他参数

```
label 字段名
error_messages 自定义报错信息
initial 默认值
required 控制字段是否必填
    required=False 字段不是必须填写内容
widget 控制标签样式属性
    widget=forms.widgets.TestInput(attrs={'class':'form-control', 'username':'lxt'})  # type是文本输入框，在attr中添加属性和自定义属性
    widget=forms.widgets.PasswoordInput(attrs={'class':'form-control c1 c2'}) # 添加多个css样式
    # 多个属性值，直接空格隔开即可
    
    
validators 第一道关卡中，支持正则校验数据
from django.core.validators import RegexValidator
phone = forms.CharField(
    validators=[RegexValidator(r'^[0-9]+', '请输入数字'),
            RegexValidator(r'^159[0-9]+', '数字必须以159开头')
        ]
)


```

其他类型渲染

```
radio 
gender = forms.ChoiceField(
    choices=((1,'男'),(2,'女'),(3, '保密')),
    lable="性别",
    initial=3,
    widget=forms.widgets.RadioSelect()
)

select
gender = forms.ChoiceField(
    choices=((1,'男'),(2,'女'),(3, '保密')),
    lable="性别",
    initial=3,
    widget=forms.widgets.Select()
)

# 多选select
hobby = forms.MutipleChoiceField(
    choices=((1,"篮球"), (2,"足球"), (3,"双色球")),
    lable="爱好",
    initial=[1,3],
    widget=forms.widgets.SelectMutiple()
)

# 单选checkbox
keep = forms.ChoiceField(
    lable="是否记住密码",
    initial="checked",
    widget=forms.widgets.CheckboxInput()
)

# 多选checkbox
hobby = forms.MutipleChoiceField(
    choices=((1,"篮球"), (2,"足球"), (3,"双色球")),
    lable="爱好",
    initial=[1,3],
    widget=forms.widgets.CheckboxSelectMutiple()
)
```